# DicoCo

Marius Savart

Cette version est celle avec librosa car sur mybinder je ne peux pas intégrer mon projet avec librosa

In [36]:
# import necessaire à la création des widgets
import IPython
from IPython.display import display, clear_output, HTML, Markdown, Audio 
import ipywidgets as widgets
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
#%matplotlib ipympl

In [37]:
# definition du widget permettant de taper un texte 
textTarget = widgets.Text(
       value="",
       description='filtre hwd',
       layout={'width': '300px'} )

In [38]:
def getWordInfos(word):
    urlTarget = "https://www.linguee.fr/francais-anglais/search?source=auto&query="+word
    response = requests.get(urlTarget)
    soup = BeautifulSoup(response.text,'html.parser')
    souplettesBody = soup.find('div')
    word = souplettesBody.findAll('div',{"class":"isForeignTerm"})

    files = souplettesBody.findAll('a',{"class":"audio"})
    word = [v.find("a").text.strip() for v in word][0]
    files_split = files[0]["onclick"].replace("playSound","").replace(";","").replace("\"", "").split(",")
    files_split = {files_split[1].split("/")[0]:"https://www.linguee.fr/mp3/"+files_split[1]+".mp3",files_split[3].split("/")[0]:"https://www.linguee.fr/mp3/"+files_split[3]+".mp3"}
    return word, files_split

In [39]:
# définition du widget button permettant de lancer l'extraction
button = widgets.Button(description='Démarrer')

rangeSlideBar = widgets.IntRangeSlider(
    value=[0, 10],
    min=0,
    max=10,
    step=1,
    description='Nombres syllabes:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

slideBar = widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Max échantillons:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# définition du widget permettant l'affichage d'une sortie
out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()
out5 = widgets.Output()

cheminDuFichierCSV = "./../data/dataframe/new_dataframe.csv" 

# demander à pandas de lire le fichier et garder ce tableau accessible dans une variable
df = pd.read_csv(cheminDuFichierCSV, delimiter=";", encoding="utf8")
df = df.replace(np.nan, '', regex=True)
df = df[df.headWord.isin([""]) == False]

# spécifier les actions à réaliser lors de l'appui sur le bouton
def on_button_clicked(_):
    
    # récupérer le texte saisi par l'utilisateur
    targetWord = textTarget.value
    nbSyll = rangeSlideBar.value
    
    df_target = df.copy()
    
    with out1:
        clear_output()
        df_target = df_target[df_target["headWord"].str.contains(targetWord)]
        df_target = df_target[(df_target["nbSyll"] >= nbSyll[0]) & (df_target["nbSyll"] <= nbSyll[1])]
        if slideBar.value == 0:

            word, files = getWordInfos(df_target["headWord"][df_target.index.tolist()[0]])
            display(df_target)
        else:
            df_target = df_target.sample(n=slideBar.value)
            word, files = getWordInfos(df_target["headWord"][df_target.index.tolist()[0]])
            display(df_target)
    with out2:
        clear_output()
        display("nombres occurences ",len(df_target))
    
    with out3:
        clear_output()
        url = files["EN_UK"]
        r = requests.get(url, allow_redirects=True)

        localPath = "./../data/audio/1.mp3"
        open(localPath, 'wb').write(r.content)
        display(Audio(filename=localPath))
    with out4:
        clear_output()
        url = files["EN_US"]
        r = requests.get(url, allow_redirects=True)

        localPath = "./../data/audio/2.mp3"
        open(localPath, 'wb').write(r.content)
        display(Audio(filename=localPath))
        
button.on_click(on_button_clicked)

In [40]:
df["nbSyll"][0] >4

False

In [41]:
def returnDf():
    # préciser le chemin du fichier à lire
    cheminDuFichierCSV = "./../data/dictionary/1757_Buchanan-J.csv" 

    # demander à pandas de lire le fichier et garder ce tableau accessible dans une variable
    dfDicoBuch = pd.read_csv(cheminDuFichierCSV, delimiter=";", encoding="utf8")
    
    dfLetters = pd.read_csv("./../data/dictionary/Buchanan_PronChar_counts.csv", delimiter=";", encoding="utf8")
    dfLetters = dfLetters.replace(np.nan, '', regex=True)
    dfDicoBuch["headWord"]= None
    
    for i in range(len(dfDicoBuch)):
        word = dfDicoBuch["pron"][i]
        for char in list(dfLetters["char"]):
            word = word.replace(char, dfLetters["graph"][dfLetters["char"]==char][dfLetters["graph"][dfLetters["char"]==char].index[0]])
        dfDicoBuch["headWord"][i] = word
    return dfDicoBuch

In [42]:
def getDf(word, nbSyll):
    # récupérer le texte saisi par l'utilisateur
    targetWord = textTarget.value
    df_target = df.copy()
    with out1:
        clear_output()
        df_target = nbSyll[0] <= df_target[df_target["headWord"].str.match(word).str.len() <= nbSyll[1]]
        display(df_target)
    with out2:
        clear_output()
        display(len(df_target))

In [43]:
# les widgets de filtre input
vbFiltres = widgets.HBox([textTarget, rangeSlideBar, slideBar, button])

# les widgets d'output
vbAudio = widgets.HBox([out3, out4])
vbOuts = widgets.VBox([out1,out2,vbAudio, out5])

hbAllWidgets = widgets.VBox([vbFiltres,vbOuts])
hbAllWidgets